In [38]:
#!pip install --user nteract-scrapbook

In [39]:
import os
import ipyvuetify as v
import scrapbook as sb
import yaml

In [40]:
with open('config.yml', 'r') as f:
    config = yaml.safe_load(f)

In [41]:
button_list = {
    'LINK': 'http://www.google.com',
}
my_dashboard_name = 'Viola Dashboard Server'
voila_base_url = config['server_url'] + '/voila/render/'
additional = []

In [17]:
additional = [
    {'title': 'Another Link',
      'description': 'This is a link from the additional config.',
      'link': 'https://github.com'}
]

In [18]:
lst = [
    v.Btn(children=[name], flat=True, href=link, target='_blank')
    for name, link in button_list.items()
]

In [26]:
all_files = [f for f in os.listdir('.') if os.path.isfile(f)]
all_files.remove('homepage.ipynb')

In [30]:
files = []
for file in all_files:
    extension = file.split('.')[-1]
    if extension == 'ipynb':
        files.append(file)

In [31]:
items = []
for i, f in enumerate(files):
    this_item = {}
    this_item['title'] = f
    this_item['description'] = 'A Jupyter Notebook.'
    this_item['link'] = voila_base_url + f

    nb = sb.read_notebook(f)
    for key, value in nb.scraps.data_dict.items():
        this_item[key] = value
    items.append(this_item)

In [71]:
items.extend(additional)

In [73]:
toolbar = v.Toolbar(
    children=[
        v.ToolbarTitle(children=[my_dashboard_name]),
        v.Spacer(), # Fills empty space
        v.ToolbarItems(children=lst)
    ],
    app=True
)

In [74]:
cards = [
    v.Flex(ma_2=True, children=[
        v.Card(elevation=5,
               align_center=True,
               min_height='200px',
               min_width='200px',
               max_height='200px',
               max_width='200px',
               href=details['link'],
               target='_blank',
               children=[
                   v.CardTitle(children=[details['title']]),
                   v.CardText(children=[details['description']]),
               ])
    ])
    for details in items
]

In [75]:
app = v.Container(
    fluid = True,
    children = [
        toolbar,
        v.Layout(pa_5=True, wrap=True, children=cards)
    ]
)

In [76]:
app

Container(children=[Toolbar(app=True, children=[ToolbarTitle(children=['Viola Dashboard Server']), Spacer(), T…